# Imports

In [1]:
# ============================================================================
# 04_results.py
# Loads and visualizes results from 03_modeling.py
# Includes model comparison, ROC/PR curves, confusion matrices, feature importance, and DCA
# ============================================================================

import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, precision_recall_curve, auc
from pathlib import Path

# Load Models and Results

In [3]:
# ============================================================================
# 1. LOAD MODELS AND RESULTS
# ============================================================================
print("="*80)
print("LOAD MODELS AND RESULTS")
print("="*80)

# Festgelegter Pfad zu deinem models-Ordner
models_dir = Path(r"C:\Users\Eyyub\Desktop\StackFuel\PortfolioProjekt\DPP-Stackfuel-Data-Science-Projekt\models")

# Lade die gespeicherten Modell-Ergebnisse
model_results = joblib.load(models_dir / "model_results.pkl")

# Lade die einzelnen Modelle (nach Name)
models = {}
for model_name in model_results.keys():
    model_path = models_dir / f"{model_name.replace(' ', '_')}_model.pkl"
    models[model_name] = joblib.load(model_path)

# Lade die Test-Labels
target_test = joblib.load(models_dir / "target_test.pkl")

print(f"Loaded models: {list(models.keys())}")
print(f"Test labels shape: {target_test.shape}")


LOAD MODELS AND RESULTS
Loaded models: ['Random Forest', 'XGBoost', 'Logistic Regression', 'Neural Network', 'SVM', 'K-Nearest Neighbors', 'Decision Tree', 'Naive Bayes']
Test labels shape: (45895,)
